<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/guides/getting-started/tutorial_pt7.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Agents with Tool Use

Tool use extends the ideas from RAG, where external systems are used to guide the response of an LLM, but by leveraging a much bigger set of tools than what’s possible with RAG. The concept of tool use leverages LLMs' useful feature of being able to act as a reasoning and decision-making engine.

While RAG enables applications that can *answer questions*, tool use enables those that can *automate tasks*.

Tool use also enables developers to build agentic applications that can take actions, that is, doing both read and write operations on an external system.

In this tutorial, you'll learn about:
- Creating tools
- Tool planning and calling
- Tool execution
- Response and citation generation
- Multi-step tool use

You'll learn these by building an onboarding assistant for new hires.

## Setup

To get started, first we need to install the `cohere` library and create a Cohere client.

In [ ]:
# pip install cohere numpy

import numpy as np
import cohere

co = cohere.Client("COHERE_API_KEY") # Get your API key: https://dashboard.cohere.com/api-keys

## Creating tools

The pre-requisite, before we can run a tool use workflow, is to set up the tools. Let's create three tools:
- `search_faqs`: A tool for searching the FAQs. For simplicity, we'll not implement any retrieval logic, but we'll simply pass a list of pre-defined documents, which are the FAQ documents we had used in the Text Embeddings section.
- `search_emails`: A tool for searching the emails. Same as above, we'll simply pass a list of pre-defined emails from the Reranking section.
- `create_calendar_event`: A tool for creating new calendar events. Again, for simplicity, we'll not implement actual event bookings, but will return a mock success event. In practice, we can connect to a calendar service API and implement all the necessary logic here.

Here, we are defining a Python function for each tool, but more broadly, the tool can be any function or service that can receive and send objects.

In [5]:
# Create the tools
def search_faqs(query):
    faqs = [
        {"text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."},
        {"text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."}
    ]
    return  {"faqs" : faqs}

def search_emails(query):
    emails = [
        {"from": "it@co1t.com", "to": "david@co1t.com", "date": "2024-06-24", "subject": "Setting Up Your IT Needs", "text": "Greetings! To ensure a seamless start, please refer to the attached comprehensive guide, which will assist you in setting up all your work accounts."},
        {"from": "john@co1t.com", "to": "david@co1t.com", "date": "2024-06-24", "subject": "First Week Check-In", "text": "Hello! I hope you're settling in well. Let's connect briefly tomorrow to discuss how your first week has been going. Also, make sure to join us for a welcoming lunch this Thursday at noon—it's a great opportunity to get to know your colleagues!"}
    ]
    return  {"emails" : emails}
    
def create_calendar_event(date: str, time: str, duration: int):
    # You can implement any logic here
    return {"is_success": True,
            "message": f"Created a {duration} hour long event at {time} on {date}"}
    
functions_map = {
    "search_faqs": search_faqs,
    "search_emails": search_emails,
    "create_calendar_event": create_calendar_event
}

The second and final setup step is to define the tool schemas in a format that can be passed to the Chat endpoint. The schema must contain the following fields: `name`, `description`, and `parameter_definitions` in the format shown below. 

This schema informs the LLM about what the tool does, and the LLM decides whether to use a particular tool based on it. Therefore, the more descriptive and specific the schema, the more likely the LLM will make the right tool call decisions.

In [6]:
# Define the tools
tools = [
    {
      "name": "search_faqs",
      "description": "Given a user query, searches a company's frequently asked questions (FAQs) list and returns the most relevant matches to the query.",
      "parameter_definitions": {
        "query": {
          "description": "The query from the user",
          "type": "str",
          "required": True
        }
      }
    }, 
    {
      "name": "search_emails",
      "description": "Given a user query, searches a person's emails and returns the most relevant matches to the query.",
      "parameter_definitions": {
        "query": {
          "description": "The query from the user",
          "type": "str",
          "required": True
        }
      }
    }, 
    {
      "name": "create_calendar_event",
      "description": "Creates a new calendar event of the specified duration at the specified time and date. A new event cannot be created on the same time as an existing event.",
      "parameter_definitions": {
        "date": {
          "description": "the date on which the event starts, formatted as mm/dd/yy",
          "type": "str",
          "required": True
        },
        "time": {
          "description": "the time of the event, formatted using 24h military time formatting",
          "type": "str",
          "required": True
        },
        "duration": {
          "description": "the number of hours the event lasts for",
          "type": "float",
          "required": True
        }
      }
    }
]

## Tool planning and calling

We can now run the tool use workflow. We can think of a tool use system as consisting of four components:
- The user
- The application
- The LLM
- The tools

At its most basic, these four components interact in a workflow through four steps:
- **Step 1: Get user message** – The LLM gets the user message (via the application)
- **Step 2: Tool planning and calling** – The LLM makes a decision on the tools to call (if any) and generates - the tool calls
- **Step 3: Tool execution** - The application executes the tools and the results are sent to the LLM
- **Step 4: Response and citation generation** – The LLM generates the response and citations to back to the user

In [15]:
# Step 1: Get user message
message = "Any messages about getting setup with IT?"

preamble="""## Task & Context
You are an assistant who assist new employees of Co1t with their first week. You respond to their questions and assist them with their needs. Today is Monday, June 24, 2024"""

# Step 2: Tool planning and calling
response = co.chat(
    message=message,
    preamble=preamble,
    tools=tools)

if response.tool_calls:
    print("Tool plan:")
    print(response.text,"\n")
        
    print("Tool calls:")
    for call in response.tool_calls:
        print(f"Tool name: {call.name} | Parameters: {call.parameters}")

Tool plan:
I will search the user's emails for any messages about getting set up with IT. 

Tool calls:
Tool name: search_emails | Parameters: {'query': 'IT setup'}


Given three tools to choose from, the model is able to pick the right tool (in this case, `search_emails`) based on what the user is asking for.

Also, notice that the model first generates a plan about what it should do ("I will do ...") before actually generating the tool call(s).

# Tool execution

In [12]:
# Step 3: Tool execution
tool_results = []
for tc in response.tool_calls:
    tool_call = {"name": tc.name, "parameters": tc.parameters}
    tool_output = functions_map[tc.name](**tc.parameters)
    tool_results.append({"call": tool_call, "outputs": [tool_output]})

print("Tool results:") 
for result in tool_results:
    print(result)

Tool results:
{'call': {'name': 'search_emails', 'parameters': {'query': 'IT setup'}}, 'outputs': [{'emails': [{'from': 'it@co1t.com', 'to': 'david@co1t.com', 'date': '2024-06-24', 'subject': 'Setting Up Your IT Needs', 'text': 'Greetings! To ensure a seamless start, please refer to the attached comprehensive guide, which will assist you in setting up all your work accounts.'}, {'from': 'john@co1t.com', 'to': 'david@co1t.com', 'date': '2024-06-24', 'subject': 'First Week Check-In', 'text': "Hello! I hope you're settling in well. Let's connect briefly tomorrow to discuss how your first week has been going. Also, make sure to join us for a welcoming lunch this Thursday at noon—it's a great opportunity to get to know your colleagues!"}]}]}


## Response and citation generation

In [16]:
# Step 4: Response and citation generation
response = co.chat(
    message="", # In response generation, we set the message as empty
    preamble=preamble,
    tools=tools,
    tool_results=tool_results,
    chat_history=response.chat_history
)

# Print final response
print("Final response:")
print(response.text)
print("="*50)

# Print citations (if any)
if response.citations:
    print("\nCITATIONS:")
    for citation in response.citations:
        print(citation)

    print("\nCITED REFERENCES:")
    for document in response.documents:
        print(document)

Final response:
You have an email from IT with a comprehensive guide attached to help you set up your work accounts.

CITATIONS:
start=12 end=25 text='email from IT' document_ids=['search_emails:0:2:0']
start=33 end=61 text='comprehensive guide attached' document_ids=['search_emails:0:2:0']
start=74 end=99 text='set up your work accounts' document_ids=['search_emails:0:2:0']

CITED REFERENCES:
{'emails': '[{"date":"2024-06-24","from":"it@co1t.com","subject":"Setting Up Your IT Needs","text":"Greetings! To ensure a seamless start, please refer to the attached comprehensive guide, which will assist you in setting up all your work accounts.","to":"david@co1t.com"},{"date":"2024-06-24","from":"john@co1t.com","subject":"First Week Check-In","text":"Hello! I hope you\'re settling in well. Let\'s connect briefly tomorrow to discuss how your first week has been going. Also, make sure to join us for a welcoming lunch this Thursday at noon—it\'s a great opportunity to get to know your colleagues

# Multi-step tool use

The model can execute more complex tasks in tool use – tasks that require tool calls to happen in a sequence. This is referred to as "multi-step" tool use. 

Let's create a function to called `run_assistant` to implement these steps, and along the way, print out the key events and messages. Optionally, this function also accepts the chat history as an argument to keep the state in a multi-turn conversation. 

In [18]:
model = "command-r-plus"

preamble="""## Task & Context
You are an assistant who assists new employees of Co1t with their first week. You respond to their questions and assist them with their needs. Today is Monday, June 24, 2024"""

# A function that runs multi-step tool use
def run_assistant(message, chat_history=None):
    
    if chat_history is None:
        chat_history = []
        
    # Step 1: get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)    
    response = co.chat(
        message=message,
        model=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history
    )

    # Tool execution loop
    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Intermediate response:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("\nCITATIONS:")
        for citation in response.citations:
            print(citation)

        print("\nCITED REFERENCES:")
        for document in response.documents:
            print(document)
                
    return chat_history

To illustrate the concept of multi-step tool user, let's ask the assistant to block time for any lunch invites received in the email.

This requires tasks to happen over multiple steps in a sequence. Here, we see the assistant running these steps:
- First, it calls the `search_emails` tool to find any lunch invites, which it found one.
- Next, it calls the `create_calendar_event` tool to create an event to block the person's calendar on the day mentioned by the email.

This is also an example of tool use enabling a write operation instead of just a read operation that we saw with RAG.

In [19]:
chat_history = run_assistant("Can you check if there are any lunch invites, and for those days, block an hour on my calendar from 12-1PM.")

Question:
Can you check if there are any lunch invites, and for those days, block an hour on my calendar from 12-1PM.
Intermediate response:
I will search the user's emails for lunch invites, and then create calendar events for the dates and times of those invites. 

Tool calls:
Tool name: search_emails | Parameters: {'query': 'lunch invite'}
Intermediate response:
I have found one lunch invite for Thursday 27 June at noon. I will now create a calendar event for this. 

Tool calls:
Tool name: create_calendar_event | Parameters: {'date': '06/27/24', 'duration': 1, 'time': '12:00'}
Final response:
I found one lunch invite for Thursday 27 June at noon. I have created a calendar event for this.

CITATIONS:
start=29 end=53 text='Thursday 27 June at noon' document_ids=['search_emails:0:2:0']
start=62 end=95 text='created a calendar event for this' document_ids=['create_calendar_event:0:4:0']

CITED REFERENCES:
{'emails': '[{"date":"2024-06-24","from":"it@co1t.com","subject":"Setting Up Your 

In this tutorial, you learned about:
- How to create tools
- How tool planning and calling happens
- How tool execution happens
- How to generate the response and citations
- How to run tool use in a multi-step scenario

And that concludes our 7-part Cohere tutorial. We hope that they have provided you with a foundational understanding of the Cohere API, the available models and endpoints, and the types of use cases that you can build with them.

To continue your learning, check out:
- [LLM University - A range of courses and step-by-step guides to help you start building](https://cohere.com/llmu)
- [Cookbooks - A collection of basic to advanced example applications](https://docs.cohere.com/page/cookbooks)
- [Cohere's documentation](https://docs.cohere.com/docs/the-cohere-platform)
- [The Cohere API reference](https://docs.cohere.com/reference/about)